# Imports

In [ ]:
import os
import pathlib
import random
from math import sqrt
import ujson

import pandas as pd
import plotly.express as px

if "PATH" not in globals():
    PATH = pathlib.Path.cwd() / "src"

# set current working directory to the path
os.chdir(PATH)
print(f"Current path: {PATH}")

In [ ]:
# update the PYTHONPATH
!export PYTHONPATH=$PYTHONPATH:{PATH}

In [ ]:
from src.MOO import MOO
from src.optimizers.NSGA2 import NSGA2
from src.optimizers.NSRA import NSRA
from src.optimizers.NSWGE import NSWGE
from src.problems.multitask_routing.Network import Network
from src.problems.multitask_routing.Task import Task
from src.problems.Solution import Solution

# Defining the parameters

In [ ]:
random.seed(42)

## Layers

In [ ]:
# PROBLEM DEFINITION
paramsLayerOne = {
    "unit": {
        "tag": "DEVICE",
        "computingSpeed": lambda: random.randint(5, 10),
        "positionX": 0,
        "positionY": 0,
        "throughput": 0.5,
        "pollution": lambda: round(random.random(), 2),
        "cost": lambda: round(random.random(), 2),
    },
    "cable": {
        "distance": lambda x, y: sqrt(
            pow(x.positionX - y.positionX, 2)
            + pow(x.positionY - y.positionY, 2)
        ),
        "propagationSpeed": lambda: 1,
        "flowRate": lambda: 1,
    },
    "numberNewUnits": 10,
}

paramsLayerTwo = {
    "unit": {
        "tag": "FOG",
        "computingSpeed": lambda x: x.computingSpeed * random.randint(5, 10),
        "positionX": lambda x: x.positionX + random.randint(-2, 2),
        "positionY": lambda x: x.positionY + random.randint(-2, 2),
        "throughput": lambda x: x.throughput * round(random.random(), 2) * 2
        + 1,
        "pollution": lambda x: (x.pollution + 1)
        * (round(random.random(), 2) + 1),
        "cost": lambda x: (x.cost + 1) * (round(random.random(), 2) + 1),
    },
    "cable": {
        "distance": lambda x, y: sqrt(
            pow(x.positionX - y.positionX, 2)
            + pow(x.positionY - y.positionY, 2)
        ),
        "propagationSpeed": lambda: 2,
        "flowRate": lambda: 3,
    },
    "numberNewUnits": 20,
}

paramsLayerThree = {
    "unit": {
        "tag": "CLOUD",
        "computingSpeed": lambda x: x.computingSpeed * random.randint(50, 100),
        "positionX": lambda x: x.positionX + random.randint(-5, 5),
        "positionY": lambda x: x.positionY + random.randint(-5, 5),
        "throughput": lambda x: x.throughput * round(random.random(), 2) * 3
        + 1,
        "pollution": lambda x: (x.pollution + 1)
        * (round(random.random(), 2) + 1),
        "cost": lambda x: (x.cost + 1) * (round(random.random(), 2) + 1),
    },
    "cable": {
        "distance": lambda x, y: sqrt(
            pow(x.positionX - y.positionX, 2)
            + pow(x.positionY - y.positionY, 2)
        ),
        "propagationSpeed": lambda: 2,
        "flowRate": lambda: 3,
    },
    "numberNewUnits": 10,
}

## Tasks

In [ ]:
tasks = tuple(
    Task(random.randint(10, 100000), random.randint(10, 100000)) for _ in range(4)
)

## Network

In [ ]:
optimUnits = {
    "processingTime": "Processing time (s)",
    "cost": "Cost (€)",
    "pollution": "Pollution (g)",
}
optimDirections = {
    "processingTime": "min",
    "cost": "min",
    "pollution": "min",
}
problem = Network(
    "DEVICE",
    tasks=tasks,
    optimDirections=optimDirections,
    minDepth=1,
    maxDepth=15,
    mutationRate=0.1,  # it should be an algorithm parameter
    layers=[paramsLayerOne, paramsLayerTwo, paramsLayerTwo, paramsLayerThree],
)

## Optimization

In [ ]:
nIterations = 10
nSolutions = 100

# Optimization

## Training

In [ ]:
moo = MOO(problem)

In [ ]:
nswge_paretos = moo.optimize(
    NSWGE,
    nSolutions,
    nIterations,
    imDir=PATH.parent / "imgs",
    saveDir=PATH.parent / "paretos",
    export=True,
)

In [ ]:
nsga2_paretos = moo.optimize(
    NSGA2,
    nSolutions,
    nIterations,
    imDir=PATH.parent / "imgs",
    saveDir=PATH.parent / "paretos",
    export=True,
    ratioKept=0.5,
)

In [ ]:
nsra_paretos = moo.optimize(
    NSRA,
    nSolutions,
    nIterations,
    imDir=PATH.parent / "imgs",
    saveDir=PATH.parent / "paretos",
    export=True,
    ratioKept=0.5,
)

## Evaluation

In [ ]:
MOO.relative_efficiency(nsra_paretos, nsga2_paretos, Solution.optimDirections, verbose=True)
MOO.relative_efficiency(nswge_paretos, nsga2_paretos, Solution.optimDirections, verbose=True)
MOO.relative_efficiency(nswge_paretos, nsra_paretos, Solution.optimDirections, verbose=True)


# Results

In [ ]:
def generate_violin_plot(df, yaxis_title="Value"):
    fig = px.violin(
        df,
        x="Epoch",
        y="Value",
        box=True,
        points=False,
        hover_data=df.columns,
        template="plotly_white",
        color_discrete_sequence=["#1f77b4"],
    )
    fig.update_layout(
        xaxis_title="Epoch",
        yaxis_title=yaxis_title,
        font=dict(
            family="Times New Roman",
            size=12,
        ),
    )
    fig.show()

In [ ]:
# load pareto data from json
for file in os.listdir(PATH.parent / "paretos"):
    if file.endswith(".json"):
        # load file as dict
        print(file)
        with open(PATH.parent / "paretos" / file, "r") as f:
            data = ujson.load(f)
        for objective in optimDirections.keys():
            # unstack values
            df = (
                pd.DataFrame.from_dict(data[objective], orient="index")
                .stack()
                .reset_index()
                .rename(columns={"level_0": "Epoch", 0: "Value"})
                .drop(columns="level_1")
            )
            # generate plot
            generate_violin_plot(
                df,
                yaxis_title=optimUnits[objective],
            )
